# Story Metrics

In [ ]:
# Connect to Trino

import yaml
import pandas as pd
from sqlalchemy import create_engine


yaml_connection_file = open("C://Users/mario/OneDrive/Documentos/utilities/trino_db.yaml")
yaml_dict = yaml.load(yaml_connection_file, Loader=yaml.FullLoader)
locals().update(yaml_dict['presto'])


db = '/hive/dm'
conn_string = 'presto://' + username + ':' + str(password) + '@' + host + ':' + str(port) + db

engine = create_engine(conn_string, connect_args={'protocol': protocol,
                          'session_props': {'query_max_run_time': '1234m'}})

## Exports

In [2]:
import xlwings as xw

def writes_df_excel(sht, df, rangename):
    # clears existing data
    x0 = sht.range('A1').end('down')
    y0 = sht.range('A1').end('right')
    sht.range(x0,y0).clear()

    sht.range('A1').options(index=False).value  = df
    tbl_range = sht.range("A1").expand('table')
    sht.api.ListObjects.Add(1, sht.api.Range(tbl_range.address))
    
    sht.range(tbl_range).expand().name = rangename
    
wb = xw.Book(r'C://Users/mario/OneDrive/Documentos/Data Science/research/Content Acq/story_metrics.xlsx')


# Story Metrics

In [2]:
# Filter Stories
stories = (16826,
            21450,
            16539,
            10886,
            16991,
            21562,
            25343,
            21564,
            18340,
            25594,
            21544,
            25503,
           21965,
           16539
)

# some stories have previous chapters launched early; we count since promotion dates
launch_dates = (
    '2022-07-05',
    '2022-07-11',
    '2022-07-11',
    '2022-07-18',
    '2022-07-21',
    '2022-07-22',
    '2022-08-12',
    '2022-08-19',
    '2022-09-03',
    '2022-09-06',
    '2022-09-20',
    '2022-10-04',
    '2022-10-10',
    '2022-10-17'
)

# periods in days to analyze Story performance
periods = (30,60,90)

In [ ]:
# Creates Hard Coded Select Statements for input stories
sel_clause_stories = ""
sel_clause_activeusers = ""

for stid, ld in zip(stories, launch_dates):
    sel_clause_stories += f"""(select {stid} as story_id, Date '{ld}' as launch_date FROM MySQL.gatsby.stories LIMIT 1) UNION ALL """
    
    for per in periods:
        sel_clause_activeusers += f"""select {stid} as story_id, Date '{ld}' as launch_date, {per} as period, count(distinct user_id) as active_users
                                  from hive.dm.active_users_utc
                                  where base_date >=  Date '{ld}' and base_date <= DATE_ADD('day', {per} ,  Date '{ld}')
                                  group by 1, 2,3 UNION ALL """
print(sel_clause_activeusers)
    
    

# remove the last UNION ALL
sel_clause_stories = sel_clause_stories[:-10]
sel_clause_activeusers = sel_clause_activeusers[:-10]


In [18]:
df_stories_mt = pd.DataFrame()

for per in periods:
    for stid, ld in zip(stories, launch_dates):
        q_story_mt = f"""
        with stories as (
        SELECT id as story_id, title as story_title, Date '{ld}' as launch_date
        FROM MySQL.gatsby.stories
        WHERE status = 'published'
        AND id = {stid}
        ),

        monthly_revenue_by_story AS (
        SELECT s.story_id, story_title,  s.launch_date,
          sum( case when is_paid_coin = 1 then coin_amount else 0 end) as paid_coins,
          sum( case when is_paid_coin = 0 then coin_amount else 0 end) as free_coins,
          count( case when is_paid_coin = 1 then episode_id else null end) as paid_eps,
          count( case when is_paid_coin = 0 then episode_id else null end) as free_eps
        from stories s
        left outer join hive.dm.coins_user_episode cue
          on s.story_id = cue.story_id
            and cue.used_date >= s.launch_date
            and cue.used_date <= DATE_ADD('day',{per} , s.launch_date)
        group by 1,2,3
        ),

        conversion as (
        select s.story_id, 
               count(distinct sc.user_id) as viewers, count(distinct sc.viewed_at) as impressions, 
               count(distinct pressed_at) as clicks, count(distinct read_at) as reads
        from stories s
        join hive.dm.story_conversion sc
          on s.story_id = sc.story_id
        where base_date >= s.launch_date
        and base_date <= DATE_ADD('day',{per} , s.launch_date)
        group by 1
        ),

        readers_by_story AS (
        select s.story_id, rbe.user_id, 
         count(*) as story_total_reads
        from stories s
         join hive.dm.readers_by_episode rbe
          on s.story_id = rbe.story_id
         where rbe.read_date >= s.launch_date
         and rbe.read_date <= DATE_ADD('day',{per} , s.launch_date)
         group by 1,2
        ),


        moments as (
         select s.story_id, 
            count(distinct rbs.user_id) as story_unique_readers,
            count(distinct sm.user_id) as set_up_readers,
            count(distinct am.user_id) as aha_readers,         count(distinct hm.user_id) as habit_readers

         from stories s
         join readers_by_story rbs
         on s.story_id = rbs.story_id

         left outer join hive.dm.setup_moment sm
          on s.story_id = sm.story_id
          and sm.join_date >= s.launch_date

         left outer join hive.dm.aha_moment am
          on rbs.user_id = am.user_id
          and am.d3_read_days = 3
          and am.join_date >= s.launch_date

        left outer join hive.dm.habit_moment hm
          on rbs.user_id = hm.user_id
          and hm.d7_read_days >= 5  
          and hm.join_date >= s.launch_date
          group by 1
          ),
           
        active_users as (
                select * from ({sel_clause_activeusers})
                where period = {per}
        )


        Select cv.story_id, story_title,  mr.launch_date, '{per}' as period,
               paid_coins, free_coins, paid_eps, free_eps,
               viewers, impressions, clicks, reads,
               story_unique_readers,
               set_up_readers, aha_readers, habit_readers,
               active_users

        from monthly_revenue_by_story mr
        left outer join conversion cv
          on cv.story_id = mr.story_id
        left outer join moments m
          on cv.story_id = m.story_id
        left outer join active_users aup
            on cv.story_id = aup.story_id

        """
     #   df_stories_mt0 = pd.read_sql(q_story_mt, engine)

      #  df_stories_mt = pd.concat([df_stories_mt, df_stories_mt0])
        print(q_story_mt)


        with stories as (
        SELECT id as story_id, title as story_title, Date '2022-07-05' as launch_date
        FROM MySQL.gatsby.stories
        WHERE status = 'published'
        AND id = 16826
        ),

        monthly_revenue_by_story AS (
        SELECT s.story_id, story_title,  s.launch_date,
          sum( case when is_paid_coin = 1 then coin_amount else 0 end) as paid_coins,
          sum( case when is_paid_coin = 0 then coin_amount else 0 end) as free_coins,
          count( case when is_paid_coin = 1 then episode_id else null end) as paid_eps,
          count( case when is_paid_coin = 0 then episode_id else null end) as free_eps
        from stories s
        left outer join hive.dm.coins_user_episode cue
          on s.story_id = cue.story_id
            and cue.used_date >= s.launch_date
            and cue.used_date <= DATE_ADD('day',30 , s.launch_date)
        group by 1,2,3
        ),

        conversion as (
        select s.story_id, 
               count(di

In [ ]:
df_stories_mt.head()

In [27]:
# Normalize by Impressions

df_stories_mt['PaidC_Imp'] = df_stories_mt['paid_coins']/df_stories_mt['impressions']
df_stories_mt['FreeC_Imp'] = df_stories_mt['free_coins']/df_stories_mt['impressions']

df_stories_mt['SetUp_UnqReaders'] = df_stories_mt['set_up_readers']/df_stories_mt['story_unique_readers']
df_stories_mt['Aha_UnqReaders'] = df_stories_mt['aha_readers']/df_stories_mt['story_unique_readers']
df_stories_mt['Habit__UnqReaders'] = df_stories_mt['habit_readers']/df_stories_mt['story_unique_readers']

df_stories_mt['viewers_imp'] = df_stories_mt['habit_readers']/df_stories_mt['impressions']
df_stories_mt['clicks_viewer'] = df_stories_mt['clicks']/df_stories_mt['viewers']
df_stories_mt['unqReaders_click'] = df_stories_mt['story_unique_readers']/df_stories_mt['clicks']

df_stories_mt['unqReaders_active'] = df_stories_mt['story_unique_readers']/df_stories_mt['active_users']
df_stories_mt['PaidC_unqReaders'] = df_stories_mt['paid_coins']/ df_stories_mt['story_unique_readers']

In [29]:
wb = xw.Book(r'C://Users/mario/OneDrive/Documentos/Data Science/research/Content Acq/story_metrics.xlsx')
sht = wb.sheets['Story_Metrics']
writes_df_excel(sht, df=df_stories_mt, rangename='Story_Metrics')

In [23]:
wb = xw.Book(r'C://Users/mario/OneDrive/Documentos/Data Science/research/Content Acq/story_metrics.xlsx')
sht = wb.sheets['Story_MetricsX']
writes_df_excel(sht, df=df_stories_mt, rangename='Story_MetricsX')

## code for all stories at once
my not run 

In [17]:
df_stories_mt = pd.DataFrame()

for per in periods:
    q_story_mt = f"""
    with stories_input as (
        {sel_clause_stories} 
        ),   
    stories as (
        SELECT id as story_id, title as story_title, launch_date
        FROM MySQL.gatsby.stories gst
        join stories_input tst
        on tst.story_id = gst.id
        WHERE status = 'published'
        ),
        
        monthly_revenue_by_story AS (
        SELECT s.story_id, story_title,  s.launch_date, 
          sum( case when is_paid_coin = 1 then coin_amount else 0 end) as paid_coins,
          sum( case when is_paid_coin = 0 then coin_amount else 0 end) as free_coins,
          count( case when is_paid_coin = 1 then episode_id else null end) as paid_eps,
          count( case when is_paid_coin = 0 then episode_id else null end) as free_eps
        from hive.dm.coins_user_episode cue
        join stories s
          on s.story_id = cue.story_id
        where cue.used_date >= s.launch_date
        and cue.used_date <= DATE_ADD('day',{per} , s.launch_date)
        group by 1,2,3
        ),

        conversion as (
        select s.story_id, 
               count(distinct sc.user_id) as viewers, count(distinct sc.viewed_at) as impressions, 
               count(distinct pressed_at) as clicks, count(distinct read_at) as reads
        from stories s
        join hive.dm.story_conversion sc
          on s.story_id = sc.story_id
        where base_date >= s.launch_date
        and base_date <= DATE_ADD('day',{per} , s.launch_date)
        group by 1
        ),

        readers_by_story AS (
        select s.story_id, rbe.user_id, 
         count(*) as story_total_reads
        from stories s
         join hive.dm.readers_by_episode rbe
          on s.story_id = rbe.story_id
         where rbe.join_date >= s.launch_date
         and rbe.read_date >= s.launch_date
         and rbe.read_date <= DATE_ADD('day',{per} , s.launch_date)
         group by 1,2
        ),

        moments as (
         select s.story_id, 
            count(distinct rbs.user_id) as story_unique_readers,
            count(distinct sm.user_id) as set_up_readers,
            count(distinct am.user_id) as aha_readers,
            count(distinct hm.user_id) as habit_readers

         from stories s
         join readers_by_story rbs
         on s.story_id = rbs.story_id

         left outer join hive.dm.setup_moment sm
          on s.story_id = sm.story_id
          and sm.join_date >= s.launch_date

         left outer join hive.dm.aha_moment am
          on rbs.user_id = am.user_id
          and am.d3_read_days = 3
          and am.join_date >= s.launch_date

        left outer join hive.dm.habit_moment hm
          on rbs.user_id = hm.user_id
          and hm.d7_read_days >= 5  
          and hm.join_date >= s.launch_date
          group by 1
          )

        Select cv.story_id, story_title,  mr.launch_date, '{per}' as period,
               paid_coins, free_coins, paid_eps, free_eps,
               viewers, impressions, clicks, reads,
               story_unique_readers,
               set_up_readers, aha_readers, habit_readers

        from monthly_revenue_by_story mr
        join conversion cv
          on cv.story_id = mr.story_id
        left outer join moments m
          on cv.story_id = m.story_id
        
        """
    # df_stories_mt0 = pd.read_sql(q_story_mt, engine)

    # df_stories_mt = pd.concat([df_stories_mt, df_stories_mt0])
print(q_story_mt)


    with stories_input as (
        (select 16826 as story_id, Date '2022-07-05' as launch_date FROM MySQL.gatsby.stories LIMIT 1) UNION ALL (select 21450 as story_id, Date '2022-07-11' as launch_date FROM MySQL.gatsby.stories LIMIT 1) UNION ALL (select 16539 as story_id, Date '2022-07-11' as launch_date FROM MySQL.gatsby.stories LIMIT 1) UNION ALL (select 10886 as story_id, Date '2022-07-18' as launch_date FROM MySQL.gatsby.stories LIMIT 1) UNION ALL (select 16991 as story_id, Date '2022-07-21' as launch_date FROM MySQL.gatsby.stories LIMIT 1) UNION ALL (select 21562 as story_id, Date '2022-07-22' as launch_date FROM MySQL.gatsby.stories LIMIT 1) UNION ALL (select 25343 as story_id, Date '2022-08-12' as launch_date FROM MySQL.gatsby.stories LIMIT 1) UNION ALL (select 21564 as story_id, Date '2022-08-19' as launch_date FROM MySQL.gatsby.stories LIMIT 1) UNION ALL (select 18340 as story_id, Date '2022-09-03' as launch_date FROM MySQL.gatsby.stories LIMIT 1) UNION ALL (select 25594 as s

In [9]:
2+2


4